In [220]:
import pandas as pd
pd.options.mode.chained_assignment = None
import json
from datetime import datetime


__Load JSON data__

In [221]:
d = json.load(open('../../data/ocean_data.json'))
df = pd.DataFrame(d.get('features'))[['properties']]

In [222]:
df.index = pd.to_datetime(df.properties.apply(lambda x: x.get('observed')))
df.index = df.index.tz_localize(None)
df['sea_level'] = df.properties.apply(lambda x: x.get('value'))

# assert all rows have parameterId sea_reg
df['parameter'] = df.properties.apply(lambda x: x.get('parameterId'))
assert (df.parameter == 'sea_reg').all()
df.drop(columns=['parameter', 'properties'], inplace=True)

__Find measurements by date__

In [223]:
df_sea_level = df
df_sea_level = df_sea_level.sort_index(axis=0)

# Experimental

In [228]:
df = pd.read_pickle('../../output/239-3600-we-wa.pkl', compression='gzip')

In [237]:
def get_sea_levels_row(date):
    ix = df_sea_level.index.get_indexer([date], method='nearest')[0]
    return df_sea_level.iloc[ix - 4: ix - 1].mean().iat[0],\
           df_sea_level.iloc[ix: ix + 3].mean().iat[0]

def get_sea_levels(df_part):
    df_part['tides'] = df_part.index.map(get_sea_levels_row)
    df_part['sea_level_last_30min'] = df_part.tides.apply(lambda x: x[0])
    df_part['sea_level_next_30min'] = df_part.tides.apply(lambda x: x[1])
    return df_part

pool = Pool(processes=4)


In [238]:
df

,water_level,outflow_level,currents,current_tot,pumping_station,temp,prcp,snow,clean_water_usage_day,tides,sea_level_last_30min,sea_level_next_30min
2020-04-23 12:00:00,1.466236,4.533122,"[2.1602753277777778, 0.0]",2.160275,PUMPING_STATION_ENUM.PST239,11.0,0.0,NaN,447.0,"(2.3333333333333335, 3.3333333333333335)",2.333333,3.333333
2020-04-23 13:00:00,0.000000,13.287046,"[6.301382713888889, 0.0]",6.301383,PUMPING_STATION_ENUM.PST239,12.0,0.0,NaN,447.0,"(4.0, 5.333333333333333)",4.000000,5.333333
2020-04-23 14:00:00,0.000000,6.286903,"[2.981136261111111, 0.0]",2.981136,PUMPING_STATION_ENUM.PST239,13.0,0.0,NaN,447.0,"(3.3333333333333335, 6.0)",3.333333,6.000000
2020-04-23 15:00:00,0.000000,14.812008,"[6.953821866666667, 0.0]",6.953822,PUMPING_STATION_ENUM.PST239,12.0,0.0,NaN,447.0,"(5.333333333333333, 5.333333333333333)",5.333333,5.333333
2020-04-23 16:00:00,0.000000,14.750702,"[6.869975986111111, 0.0]",6.869976,PUMPING_STATION_ENUM.PST239,12.0,0.0,NaN,447.0,"(5.666666666666667, 6.0)",5.666667,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-23 07:00:00,0.757506,22.834174,"[10.967703544444445, 0.0]",10.967704,PUMPING_STATION_ENUM.PST239,5.3,0.0,NaN,NaN,"(-24.666666666666668, -26.333333333333332)",-24.666667,-26.333333
2021-02-23 08:00:00,0.784109,28.277850,"[13.413901605555555, 0.0]",13.413902,PUMPING_STATION_ENUM.PST239,5.6,0.0,NaN,NaN,"(-28.0, -27.666666666666668)",-28.000000,-27.666667
2021-02-23 09:00:00,0.748224,22.573596,"[10.810242, 0.0]",10.810242,PUMPING_STATION_ENUM.PST239,5.6,0.0,NaN,NaN,"(-29.333333333333332, -28.333333333333332)",-29.333333,-28.333333
2021-02-23 10:00:00,0.801731,29.837787,"[14.210679952777777, 0.0]",14.210680,PUMPING_STATION_ENUM.PST239,5.5,0.0,NaN,NaN,"(-27.666666666666668, -30.0)",-27.666667,-30.000000


In [227]:
from multiprocesspandas import applyparallel

def get_sea_levels(row):
    print(row)
    print("HOI")
    date = row.date
    ix = df_sea_level.index.get_indexer([date], method='nearest')[0]
    return (df_sea_level.iloc[ix-4: ix-1].mean().iat[0], df_sea_level.iloc[ix: ix+1].mean().iat[0])

df['date'] = df.index
df['tides'] = df.apply_parallel(get_sea_levels, axis=1, num_processes=4)
df['sea_level_last_30min'] = df.tides.apply(lambda x: x[0])
df['sea_level_next_30min'] = df.tides.apply(lambda x: x[1])
df.drop(columns=['tides'], inplace=True)
df

2020-04-23 12:00:00    1.466236
2020-04-23 13:00:00    0.000000
2020-04-23 14:00:00    0.000000
2020-04-23 15:00:00    0.000000
2020-04-23 16:00:00    0.000000
                         ...   
2021-02-23 07:00:00    0.757506
2021-02-23 08:00:00    0.784109
2021-02-23 09:00:00    0.748224
2021-02-23 10:00:00    0.801731
2021-02-23 11:00:00    0.805357
Name: water_level, Length: 5875, dtype: float642020-04-23 12:00:00     4.533122
2020-04-23 13:00:00    13.287046
2020-04-23 14:00:00     6.286903
2020-04-23 15:00:00    14.812008
2020-04-23 16:00:00    14.750702
                         ...    
2021-02-23 07:00:00    22.834174
2021-02-23 08:00:00    28.277850
2021-02-23 09:00:00    22.573596
2021-02-23 10:00:00    29.837787
2021-02-23 11:00:00    24.255535
Name: outflow_level, Length: 5875, dtype: float64
HOI

HOI
2020-04-23 12:00:00     2.160275
2020-04-23 13:00:00     6.301383
2020-04-23 14:00:00     2.981136
2020-04-23 15:00:00     6.953822
2020-04-23 16:00:00     6.869976
              

AttributeError: 'Series' object has no attribute 'date'

In [170]:
df_sea_level.iloc[56-4: 56-1].mean().iat[0]

2.0